In [1]:
# https://medium.com/@a173823/web-scraping-on-shopee-com-6180979eff78
# Regulatron01!
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from collections import Counter
from tqdm import tqdm
import time
import re

import pandas as pd

# import threading

# import numpy as np
# from numpy import nan
# import re
# import concurrent.futures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math

In [2]:
def converter_dict(dict_original):
    dict_convertido = {}
    for chave, valor in dict_original.items():
        if '|' in valor:
            lista_valor = valor.split('|')
            dict_convertido[chave] = lista_valor
        else:
            dict_convertido[chave] = valor
    return dict_convertido

def testar_palavras(texto_pesquisado, yes_words, no_words)-> bool:
    
    """
    Testa se as palavras de yes-words e no-words estão dentro de texto_pesquisado.

    Args:
        texto_pesquisado (str): Texto que deve vir da página web. Pode mesclar o título com a descrição do produto.
        yes_words (list): lista de palavras que fazem a função retornar True.
        no_words (list): lista de palavras que fazem a função retornar False.

    Returns:
        Bolean:
    """   
    contagem_no_words = 0
    texto_pesquisado = texto_pesquisado.lower()
    
    for palavra in yes_words:
        print('========================== yes_words ')
        if palavra in texto_pesquisado:
            return True
    
    for palavra in no_words:
        print('========================== no words ')
        if palavra in texto_pesquisado:
            contagem_no_words+= 1
    
    print ('total:', contagem_no_words)
    if contagem_no_words > 0:
        return False
    else:
        return True    

In [6]:
# create object for chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                              options = chrome_options)  
driver.implicitly_wait(5)
actions = ActionChains(driver)

# Maximiza a janela
driver.maximize_window()
# navegar até a shopee
driver.get('https://shopee.com.br/')
time.sleep(3)
elements = driver.find_elements(By.TAG_NAME, 'button')

# clica em aceitar todos os cookies
for e in elements:
    if e.text == 'Aceitar todos os cookies':
        actions.move_to_element(e)
        actions.perform()
        e.click()
        break
        
time.sleep(3)

# clica em algum lugar da página
element = driver.find_element(By.ID, 'main')
actions.move_to_element(element).click().perform()
time.sleep(3)

# clicar para entrar
e = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Entre')[0]
actions.move_to_element(e).click().perform()

# fazer login
telefone = '71999243010'
senha    = 'Regulatron01!'

campo_telefone = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[0]
campo_senha = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[1]

actions.move_to_element(campo_telefone).click().perform()
for i in telefone:
    campo_telefone.send_keys(i)
    time.sleep(0.5)

actions.move_to_element(campo_senha).click().perform()
for i in senha:
    campo_senha.send_keys(i)
    time.sleep(0.5)

# clicar para entrar
elements = driver.find_elements(By.TAG_NAME, 'button')
for e in elements:
    if e.text == 'ENTRE':
        actions.move_to_element(e).click().perform()
        break
        
time.sleep(3)

# clica em algum lugar da página
e = driver.find_element(By.CSS_SELECTOR, 'body')
actions.move_to_element(e).click().perform()
time.sleep(3)
search_box = driver.find_element(By.CLASS_NAME, 'shopee-searchbar-input__input')
actions.move_to_element(search_box).click().perform()
time.sleep(0.5)

# pesquisar
limite_de_produtos = 0
set_produtos = set() 
query = 'carregador de celular'
time.sleep(0.5)

for i in query:
    search_box.send_keys(i)
    time.sleep(0.5)

header = driver.find_element(By.TAG_NAME,'header')
button = header.find_elements(By.TAG_NAME,'button')[0]
actions.move_to_element(button).click().perform()
time.sleep(3)



## percorrer páginas
try:
    ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
except Exception as e:
    print(e)
    try:
        # <span class="shopee-mini-page-controller__total">6</span>
        ultima_pagina = int(driver.find_elements(By.CLASS_NAME,'shopee-mini-page-controller__total')[0].text)
    except Exception as e:
        print(e)
        ultima_pagina = 1

for i in range(ultima_pagina):
    if ( len(set_produtos) == limite_de_produtos ) and ( limite_de_produtos > 0 ) :
        break
    else:
        print('pesquisando pág', i+1, 'de', ultima_pagina)

        # rolagem para esperar carregar os produtos
        for i in range(50):
            try:
                driver.execute_script("window.scrollBy(0, 100)")
                time.sleep(0.5)
            except:
                break


        time.sleep(3)
        div_links = driver.find_element(By.CSS_SELECTOR, '#main > div > div.dYFPlI > div > div > div.sdzgsX > div.shopee-search-item-result > div.row.shopee-search-item-result__items')
        a_elements = div_links.find_elements(By.TAG_NAME, 'a')

        for a in a_elements:
            set_produtos.add(a.get_attribute('href'))

        button = driver.find_elements(By.CLASS_NAME, 'shopee-icon-button--right')[0]
        actions.move_to_element(button).click().perform()
        time.sleep(3)

        
dict_produtos = { # vai guardar todas as informações dos produtos
    'produto_pesquisado' : [],
    'titulo_produto' : [],
    'id_vendedor': [],
    'quantidade' : [],
    'localizacao' : [],
    'marca' : [],
    'preco_minimo': [],
    'preco_maximo': [],
    'preco_promo_minimo': [],
    'preco_promo_maximo': [],
    'detalhes_produto' : [],
    'quantidade': [],
    'descricao': [] ,
    'homologado': [],
    'plataforma' : [],
    'url' : [],
    'modelo' : []
}

faltam = len(set_produtos)

print('pesquisando produtos')

for link in set_produtos:
    print('faltam', faltam, 'produtos')
    faltam = faltam -1
    driver.get(link)
    
    time.sleep(3)

    for i in range(10):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break

    try:
        titulo_produto = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                            ).text
    except:
        titulo_produto = '-'

    try:
        id_vendedor    = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                            ).text
    except:
        id_vendedor    = '-'

    try:      
        qtde_vendida = driver.find_element(By.XPATH,
                                           '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                          ).text
    except:
        qtde_vendida = '0'

    try:
        localizacao = driver.find_element(By.XPATH,
                                            '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                         ).text
    except:
        localizacao = ''

    try:
        preco = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                   ).text

        if preco.find('-'):
            preco = preco.split('-')
            preco_minimo = preco[0][2:]
            preco_maximo = preco[-1][3:]
        else:
            preco_minimo = preco[2:]
            preco_maximo = '-'
    except:
        preco_minimo = '-'
        preco_maximo = '-'

    try:
        preco_promo = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                   ).text

        if preco_promo.find('-'):
            preco_promo = preco_promo.split('-')
            preco_promo_minimo = preco_promo[0][2:]
            preco_promo_maximo = preco_promo[-1][3:]
        else:
            preco_promo_minimo = preco_promo[2:]
            preco_promo_maximo = '-'

    except:
            preco_promo_minimo = '-'
            preco_promo_maximo = '-'


    try:
        detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                              ).get_attribute('innerHTML')
        detalhes_produto = detalhes_produto.replace('</a>','|</a>')

        lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

        # criar um objeto Beautiful Soup a partir do HTML do div
        soup = BeautifulSoup(detalhes_produto, 'html.parser')

        # remover todas as tags <label> e svg
        for label in soup.find_all(['label','svg']):
            label.extract()

        for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
            lista_valores.unwrap()

        # seleciona todas as tags <div> e extrai o texto
        lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
        detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
        detalhes_produto = converter_dict(detalhes_produto)

    except:
        detalhes_produto = '-'

    try:
        descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
    except:
        descricao = '-'

    try:
        quantidade = detalhes_produto['Estoque']

    except:
        quantidade = '0'

    try:
        modelo = detalhes_produto['Modelo']

    except:
        modelo = '-'
        
    try:
        marca = detalhes_produto['Marca']

    except:
        marca = '-'

    # if testar_palavras(f'{titulo_produto} {descricao} {detalhes_produto.values()}', yes_words, no_words):
    #     homologado = 'CANDIDATO'
    # else:
    #     homologado = ''
    homologado = ''

    dict_produtos['produto_pesquisado'].append(query)
    dict_produtos['titulo_produto'].append(titulo_produto)
    dict_produtos['id_vendedor'].append(id_vendedor)
    dict_produtos['quantidade'].append(quantidade)
    dict_produtos['localizacao'].append(localizacao)
    dict_produtos['marca'].append(marca)
    dict_produtos['preco_minimo'].append(preco_minimo)
    dict_produtos['preco_maximo'].append(preco_maximo)
    dict_produtos['preco_promo_minimo'].append(preco_promo_minimo)
    dict_produtos['preco_promo_maximo'].append(preco_promo_maximo)
    dict_produtos['detalhes_produto'].append(detalhes_produto)
    dict_produtos['descricao'].append(descricao)
    dict_produtos['homologado'].append(homologado)
    dict_produtos['plataforma'].append('Shopee')
    dict_produtos['url'].append(link)
    dict_produtos['modelo'].append(modelo)



df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}.csv', encoding='utf-8', index = False, sep =';')

pesquisando pág 1 de 17
pesquisando pág 2 de 17
pesquisando pág 3 de 17
pesquisando pág 4 de 17
pesquisando pág 5 de 17
pesquisando pág 6 de 17
pesquisando pág 7 de 17
pesquisando pág 8 de 17
pesquisando pág 9 de 17
pesquisando pág 10 de 17
pesquisando pág 11 de 17
pesquisando pág 12 de 17
pesquisando pág 13 de 17
pesquisando pág 14 de 17
pesquisando pág 15 de 17
pesquisando pág 16 de 17
pesquisando pág 17 de 17
pesquisando produtos
faltam 1020 produtos
faltam 1019 produtos
faltam 1018 produtos
faltam 1017 produtos
faltam 1016 produtos
faltam 1015 produtos
faltam 1014 produtos
faltam 1013 produtos
faltam 1012 produtos
faltam 1011 produtos
faltam 1010 produtos
faltam 1009 produtos
faltam 1008 produtos
faltam 1007 produtos
faltam 1006 produtos
faltam 1005 produtos
faltam 1004 produtos
faltam 1003 produtos
faltam 1002 produtos
faltam 1001 produtos
faltam 1000 produtos
faltam 999 produtos
faltam 998 produtos
faltam 997 produtos
faltam 996 produtos
faltam 995 produtos
faltam 994 produtos
fa

faltam 633 produtos
faltam 632 produtos
faltam 631 produtos
faltam 630 produtos
faltam 629 produtos
faltam 628 produtos
faltam 627 produtos
faltam 626 produtos
faltam 625 produtos
faltam 624 produtos
faltam 623 produtos
faltam 622 produtos
faltam 621 produtos
faltam 620 produtos
faltam 619 produtos
faltam 618 produtos
faltam 617 produtos
faltam 616 produtos
faltam 615 produtos
faltam 614 produtos
faltam 613 produtos
faltam 612 produtos
faltam 611 produtos
faltam 610 produtos
faltam 609 produtos
faltam 608 produtos
faltam 607 produtos
faltam 606 produtos
faltam 605 produtos
faltam 604 produtos
faltam 603 produtos
faltam 602 produtos
faltam 601 produtos
faltam 600 produtos
faltam 599 produtos
faltam 598 produtos
faltam 597 produtos
faltam 596 produtos
faltam 595 produtos
faltam 594 produtos
faltam 593 produtos
faltam 592 produtos
faltam 591 produtos
faltam 590 produtos
faltam 589 produtos
faltam 588 produtos
faltam 587 produtos
faltam 586 produtos
faltam 585 produtos
faltam 584 produtos


faltam 223 produtos
faltam 222 produtos
faltam 221 produtos
faltam 220 produtos
faltam 219 produtos
faltam 218 produtos
faltam 217 produtos
faltam 216 produtos
faltam 215 produtos
faltam 214 produtos
faltam 213 produtos
faltam 212 produtos
faltam 211 produtos
faltam 210 produtos
faltam 209 produtos
faltam 208 produtos
faltam 207 produtos
faltam 206 produtos
faltam 205 produtos
faltam 204 produtos
faltam 203 produtos
faltam 202 produtos
faltam 201 produtos
faltam 200 produtos
faltam 199 produtos
faltam 198 produtos
faltam 197 produtos
faltam 196 produtos
faltam 195 produtos
faltam 194 produtos
faltam 193 produtos
faltam 192 produtos
faltam 191 produtos
faltam 190 produtos
faltam 189 produtos
faltam 188 produtos
faltam 187 produtos
faltam 186 produtos
faltam 185 produtos
faltam 184 produtos
faltam 183 produtos
faltam 182 produtos
faltam 181 produtos
faltam 180 produtos
faltam 179 produtos
faltam 178 produtos
faltam 177 produtos
faltam 176 produtos
faltam 175 produtos
faltam 174 produtos


In [15]:
df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}_01.csv', encoding='utf-8', index = False, sep =';')

In [18]:
set_produtos_aux = list(set_produtos)
set_produtos_aux = set(set_produtos_aux[100:])

['https://shopee.com.br/Bloqueador-de-sinal-de-Faraday-Saco-Saco-Rfid-Blindagem-Blindagem-Sinal-de-Bloqueio-Bag-Bolsa-Para-Caso-Carteira-de-Cart%C3%A3o-de-Id-Chave-do-carro-ETE01r-i.361968468.22333800881?sp_atk=49e7afd4-f03f-4511-99f7-27658fbbccd6&xptdk=49e7afd4-f03f-4511-99f7-27658fbbccd6',
 'https://shopee.com.br/1-R%C3%A1dio-Do-Carro-Din-Fm-Est%C3%A9reo-Bluetooth-Mp3-Audio-Player-Celular-Handfree-Telefone-Celular-Digital-Usb-Sd-Com-Entrada-Aux-In-Dash-i.503659951.8790186761?sp_atk=1397a3d6-520f-43c8-bc69-6637a682a12b&xptdk=1397a3d6-520f-43c8-bc69-6637a682a12b',
 'https://shopee.com.br/2PCS-3-Antena-Bloqueador-De-Sinais-GPS-Detector-De-C%C3%A2mera-WiFi-Sem-Rastreamento-De-Posicionamento-i.191698007.18178276263?sp_atk=025ff084-2bc8-49d6-b04a-bda89a3915e4&xptdk=025ff084-2bc8-49d6-b04a-bda89a3915e4',
 'https://shopee.com.br/Rastreador-Gps-Bloqueador-Veicular-Carro-Moto-Hmaston-Dwq-01-Novidade-i.633431056.16686510278?sp_atk=aeb59448-6196-4313-9c4f-6654957d9c98&xptdk=aeb59448-6196-4313-9c

{'https://shopee.com.br/%E3%80%90-Entrega-24h-%E3%80%91-Tela-Sens%C3%ADvel-Ao-Toque-Universal-De-Alta-Defini%C3%A7%C3%A3o-7-Polegadas-2-Pacotes-Reprodutor-De-MP5-Multim%C3%ADdia-Central-Do-Carro-i.783810553.21157801503?sp_atk=2861e8a0-da52-4f3f-b0c3-68769c465c51&xptdk=2861e8a0-da52-4f3f-b0c3-68769c465c51',
 'https://shopee.com.br/%E3%80%90Car-Electronics%E3%80%91Autom%C3%B3vel-GPS-Sinal-Anti-Interfer%C3%AAncia-Bloqueador-De-Rastreamento-De-Ferramenta-Autom%C3%A1tica-12-24V-GRQfoB-ZPBrEimN-ejFGGJWo-i.301261589.19838740979?sp_atk=8278331f-3a3e-45f6-815a-4bd065e4d596&xptdk=8278331f-3a3e-45f6-815a-4bd065e4d596',
 'https://shopee.com.br/-Estoque-Pronto-Bloqueador-GPS-De-Carro-De-Sinal-Bloqueio-Blindagem-Anti-USB-Alimentado-Rachadura-Novo-10.12-i.261109400.16787500303?sp_atk=98ae04b8-db8c-49f4-9204-150d3445c69e&xptdk=98ae04b8-db8c-49f4-9204-150d3445c69e',
 'https://shopee.com.br/-Rhian-Celular-Tamanho-Grande-RF-GPS-Bloqueador-De-Sinal-Anti-Radia%C3%A7%C3%A3o-Bolsa-De-Prote%C3%A7%C3%A3o-COD-i

In [20]:
for link in set_produtos_aux:
    print('faltam', faltam, 'produtos')
    faltam = faltam -1
    driver.get(link)
    
    time.sleep(3)

    for i in range(10):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break

    try:
        titulo_produto = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                            ).text
    except:
        titulo_produto = '-'

    try:
        id_vendedor    = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                            ).text
    except:
        id_vendedor    = '-'

    try:      
        qtde_vendida = driver.find_element(By.XPATH,
                                           '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                          ).text
    except:
        qtde_vendida = '0'

    try:
        localizacao = driver.find_element(By.XPATH,
                                            '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                         ).text
    except:
        localizacao = ''

    try:
        preco = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                   ).text

        if preco.find('-'):
            preco = preco.split('-')
            preco_minimo = preco[0][2:]
            preco_maximo = preco[-1][3:]
        else:
            preco_minimo = preco[2:]
            preco_maximo = '-'
    except:
        preco_minimo = '-'
        preco_maximo = '-'

    try:
        preco_promo = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                   ).text

        if preco_promo.find('-'):
            preco_promo = preco_promo.split('-')
            preco_promo_minimo = preco_promo[0][2:]
            preco_promo_maximo = preco_promo[-1][3:]
        else:
            preco_promo_minimo = preco_promo[2:]
            preco_promo_maximo = '-'

    except:
            preco_promo_minimo = '-'
            preco_promo_maximo = '-'


    try:
        detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                              ).get_attribute('innerHTML')
        detalhes_produto = detalhes_produto.replace('</a>','|</a>')

        lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

        # criar um objeto Beautiful Soup a partir do HTML do div
        soup = BeautifulSoup(detalhes_produto, 'html.parser')

        # remover todas as tags <label> e svg
        for label in soup.find_all(['label','svg']):
            label.extract()

        for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
            lista_valores.unwrap()

        # seleciona todas as tags <div> e extrai o texto
        lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
        detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
        detalhes_produto = converter_dict(detalhes_produto)

    except:
        detalhes_produto = '-'

    try:
        descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
    except:
        descricao = '-'

    try:
        quantidade = detalhes_produto['Estoque']

    except:
        quantidade = '0'

    try:
        modelo = detalhes_produto['Modelo']

    except:
        modelo = '-'
        
    try:
        marca = detalhes_produto['Marca']

    except:
        marca = '-'

    # if testar_palavras(f'{titulo_produto} {descricao} {detalhes_produto.values()}', yes_words, no_words):
    #     homologado = 'CANDIDATO'
    # else:
    #     homologado = ''
    homologado = ''

    dict_produtos['produto_pesquisado'].append(query)
    dict_produtos['titulo_produto'].append(titulo_produto)
    dict_produtos['id_vendedor'].append(id_vendedor)
    dict_produtos['quantidade'].append(quantidade)
    dict_produtos['localizacao'].append(localizacao)
    dict_produtos['marca'].append(marca)
    dict_produtos['preco_minimo'].append(preco_minimo)
    dict_produtos['preco_maximo'].append(preco_maximo)
    dict_produtos['preco_promo_minimo'].append(preco_promo_minimo)
    dict_produtos['preco_promo_maximo'].append(preco_promo_maximo)
    dict_produtos['detalhes_produto'].append(detalhes_produto)
    dict_produtos['descricao'].append(descricao)
    dict_produtos['homologado'].append(homologado)
    dict_produtos['plataforma'].append('Shopee')
    dict_produtos['url'].append(link)
    dict_produtos['modelo'].append(modelo)



df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}_02.csv', encoding='utf-8', index = False, sep =';')

faltam 224 produtos
faltam 223 produtos
faltam 222 produtos
faltam 221 produtos
faltam 220 produtos
faltam 219 produtos
faltam 218 produtos
faltam 217 produtos
faltam 216 produtos
faltam 215 produtos
faltam 214 produtos
faltam 213 produtos
faltam 212 produtos
faltam 211 produtos
faltam 210 produtos
faltam 209 produtos
faltam 208 produtos
faltam 207 produtos
faltam 206 produtos
faltam 205 produtos
faltam 204 produtos
faltam 203 produtos
faltam 202 produtos
faltam 201 produtos
faltam 200 produtos
faltam 199 produtos
faltam 198 produtos
faltam 197 produtos
faltam 196 produtos
faltam 195 produtos
faltam 194 produtos
faltam 193 produtos
faltam 192 produtos
faltam 191 produtos
faltam 190 produtos
faltam 189 produtos
faltam 188 produtos
faltam 187 produtos
faltam 186 produtos
faltam 185 produtos
faltam 184 produtos
faltam 183 produtos
faltam 182 produtos
faltam 181 produtos
faltam 180 produtos
faltam 179 produtos
faltam 178 produtos
faltam 177 produtos
faltam 176 produtos
faltam 175 produtos
